In [1]:
import cv2
import pandas as pd
import re
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
from keras.models import load_model
from sklearn.preprocessing import LabelBinarizer

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-01-20 10:45:20.835051: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
TIMEFRAMES = [7, 14, 30, 90, 180]
PREDICTIONS = [5, 30, 90]
IMG_TYPES = ['MPA', 'CPA', 'MLC', 'API']

# Select whether to use transfer learning or not
TRANSFER = True
TEST = True
if TRANSFER:
    labels = pd.read_csv('labels/transfer_labels.csv')
elif TEST:
    labels = pd.read_csv('labels/test_labels.csv')
else:
    labels = pd.read_csv('labels/labels.csv')

In [11]:
# Crop the images using the bounding boxes
def crop_image(img_path):
    # Load the image in grayscale
    img = cv2.imread(img_path, 0)

    # Check if the image was loaded correctly
    if img is None:
        raise ValueError(f"Image at {img_path} not found. Please check the path.")

    # Use regular expression to match numbers followed by ".png" at the end of the filename
    match = re.search(r'(\d+)(?=\.png$)', img_path)
    
    # Check if we found a match
    if match:
        # Extract the number from the matched group
        number = int(match.group(1))
        
        # Check if the number is one of the specified values
        if number == 7:
            # Crop the image using the bounding rectangle
            crop = img[105:105+115, 80:80+38]
        elif number == 14:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+85]
        elif number == 30:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+132]
        elif number == 90:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+226]
        elif number == 180:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+414]
    return crop

# Example usage:
filenames = labels['Image'].values.tolist()

# Testing the function with the provided list of filenames
for name in filenames:
    try:
        cropped_image = crop_image(name)
        # Construct the new path for the cropped image
        new_path = name.replace('.png', '_cropped.png')
        # Save the cropped image
        cv2.imwrite(new_path, cropped_image)
    except ValueError as e:
        print(e)


In [12]:
# Create a new column called 'Image' that contains the path to the cropped image but only if they 
labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')

/var/folders/jx/r9zzxsjd7wxgmmn32r0l8cz40000gn/T/ipykernel_1952/666818149.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')


In [13]:
# Function to load and convert an image to grayscale
def load_image(image_path):
    # Load image in grayscale
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Unable to load image at path: {image_path}")
    return image

filenames = labels['Image'].values.tolist()

images = []

for name in filenames:
    try:
        img = load_image(name)
        images.append(img)
    except ValueError as e:
        print(e)

# Add a new column to the labels DataFrame to store the image arrays
labels['Image_Array'] = images

In [14]:
# Show the number of 1s and 0s in the dataset
print(labels['Label'].value_counts())

1    15757
0    13661
Name: Label, dtype: int64


In [15]:
# Sort the DataFrame by date
labels['Date'] = labels['Image'].str.extract(r'(\d{4}-\d{2}-\d{2})')
labels['Date'] = pd.to_datetime(labels['Date'])
labels = labels.sort_values(by='Date') 
print(labels.head())

                                                   Image  TimePrediction  \
0      images/MPA/SmallCap_2023-01-01 00:00:00_7_crop...               5   
18428  images/MLC/Russell2000_2023-01-01 00:00:00_90_...               5   
18429  images/MLC/Russell2000_2023-01-01 00:00:00_90_...              30   
18430  images/MLC/Russell2000_2023-01-01 00:00:00_90_...              90   
14724  images/MLC/SmallCap_2023-01-01 00:00:00_7_crop...               5   

        LastPrice  FuturePrice  Label  \
0       94.500000    97.029999      1   
18428  173.399994   177.880005      1   
18429  173.399994   194.490005      1   
18430  173.399994   178.479996      1   
14724   94.500000    97.029999      1   

                                             Image_Array       Date  
0      [[0, 0, 0, 0, 0, 189, 140, 36, 0, 0, 0, 0, 0, ... 2023-01-01  
18428  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2023-01-01  
18429  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2023-01-01  
18430  [[0, 0, 0

In [16]:
def get_investment_return(y_pred_binary, lastPrice, futurePrice):
    # Calculate Rate of Return RoR for each trade independently
    # Long positions
    long_ror = []
    long_trades = []
    long_investment = 0
    # Short positions
    short_ror = []
    short_trades = []
    short_investment = 0

    y_pred_binary = y_pred_binary.tolist()
    for i in range(len(y_pred_binary)):
        # Long position
        if y_pred_binary[i][0] == 1:
            long_return_i = ((futurePrice[i] - lastPrice[i])/lastPrice[i])
            long_ror.append(long_return_i)
            long_trades.append(100*(long_return_i))
            long_investment += 100
        # Short position
        else:
            short_return_i = ((lastPrice[i] - futurePrice[i])/lastPrice[i])
            short_ror.append(short_return_i)
            short_trades.append(100*(short_return_i))
            short_investment += 100
    
    if long_investment > 0:
        # Calculate average long RoR
        long_avg_ror = np.mean(long_ror)
        # Sum all long trades
        long_trades = np.sum(long_trades)
    else:
        long_avg_ror = 0
        long_trades = 0
    
    if short_investment > 0:
        # Calculate average short RoR
        short_avg_ror = np.mean(short_ror)
        # Sum all short trades
        short_trades = np.sum(short_trades)
    else:
        short_avg_ror = 0
        short_trades = 0

    return long_avg_ror, long_trades, long_investment, short_avg_ror, short_trades, short_investment

In [17]:
evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Long_Average_RoR', 'Long_Investment', 'Long_Return', 'Short_Average_RoR', 'Short_Investment', 'Short_Return', 'Total_Investment', 'Total_Return'])

TIMEFRAMES = [7, 14, 30, 90, 180]
PREDICTIONS = [5, 30, 90]
IMG_TYPES = ['MPA', 'CPA', 'MLC', 'API']

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction <= timeframe:
                print(f"Evaluating model predicting {prediction} days ahead using {img_type} images with {timeframe} days timeframe.")

                # Filter the data
                data = labels[(labels['TimePrediction'] == prediction) &
                              (labels['Image'].str.contains(f'/{img_type}/')) &
                              (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)

                # Start at same index as test data
                #split_index = int(len(data) * 0.8)
                #data = data[split_index:]

                # Load the model
                model_filename = f"models/noWeights_{img_type}_{timeframe}_{prediction}.h5"
                model = load_model(model_filename)

                X = np.array(data['Image_Array'].tolist()) / 255.0
                lastPrice = data['LastPrice'].tolist()
                futurePrice = data['FuturePrice'].tolist()
                y = data['Label'].values

                # Evaluate the model on test data
                y_pred = model.predict(X)
                # Convert predictions to binary: if > 0.5 then 1 else 0
                y_pred_binary = np.where(y_pred > 0.5, 1, 0)  
                
                accuracy = accuracy_score(y, y_pred_binary)
                precision = precision_score(y, y_pred_binary)
                recall = recall_score(y, y_pred_binary)
                f1_score = fbeta_score(y, y_pred_binary, beta=1)

                y_test_array = y.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                hit_rate = correct_predictions / len(y_test_array)

                long_avg_ror, long_trades, long_investment, short_avg_ror, short_trades, short_investment = get_investment_return(y_pred_binary, lastPrice, futurePrice)
                # Total investment
                total_investment = long_investment + short_investment
                total_returns = long_trades + short_trades
                
                
                print("Evaluation Metrics:")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1_score}")
                print(f"Hit Rate: {hit_rate}")
                print(f"Average Long RoR: {long_avg_ror}")
                print(f"Long Investment: {long_investment}")
                print(f"Long Return: {long_trades}")
                print(f"Average Short RoR: {short_avg_ror}")
                print(f"Short Investment: {short_investment}")
                print(f"Short Return: {short_trades}")
                print(f"Total Investment: {total_investment}")
                print(f"Total Return: {total_returns}")

                
                # Add the evaluation metrics to the DataFrame
                evaluation_df = evaluation_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Accuracy': accuracy,
                    'Precision': precision,
                    'Recall': recall,
                    'F1_Score': f1_score,
                    'Hit_Rate': hit_rate,
                    'Long_Average_RoR': long_avg_ror,
                    'Long_Investment': long_investment,
                    'Long_Return': long_trades,
                    'Short_Average_RoR': short_avg_ror,
                    'Short_Investment': short_investment,
                    'Short_Return': short_trades, 
                    'Total_Investment': total_investment,
                    'Total_Return': total_returns
                }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
if TRANSFER:
    evaluation_df.to_csv('evaluation/separate/transfer_evaluation_scores.csv', index=False)
    print("Evaluation scores saved to 'evaluation/separate/tansfer_evaluation_scores.csv'.")
else:
    evaluation_df.to_csv('evaluation/separate/test_evaluation_scores.csv', index=False)
    print("Evaluation scores saved to 'evaluation/separate/test_evaluation_scores.csv'.")

Evaluating model predicting 5 days ahead using MPA images with 7 days timeframe.
31/31 [==============================] - 2s 57ms/step
Evaluation Metrics:
Accuracy: 0.8042813455657493
Precision: 0.8008048289738431
Recall: 0.8105906313645621
F1 Score: 0.805668016194332
Hit Rate: 0.8042813455657493
Average Long RoR: 0.021451384701844888
Long Investment: 49700
Long Return: 1066.133819681691
Average Short RoR: 0.015053969851823917
Short Investment: 48400
Short Return: 728.6121408282777
Total Investment: 98100
Total Return: 1794.7459605099687
Evaluating model predicting 5 days ahead using MPA images with 14 days timeframe.
30/30 [==============================] - 4s 144ms/step
Evaluation Metrics:
Accuracy: 0.69375
Precision: 0.872093023255814
Recall: 0.46296296296296297
F1 Score: 0.6048387096774194
Hit Rate: 0.69375
Average Long RoR: 0.02806204762080435
Long Investment: 25800
Long Return: 724.0008286167522
Average Short RoR: 0.0053961474411376165
Short Investment: 70200
Short Return: 378.80

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


31/31 [==============================] - 2s 64ms/step
Evaluation Metrics:
Accuracy: 0.8409785932721713
Precision: 0.7975133214920072
Recall: 0.9144602851323829
F1 Score: 0.8519924098671727
Hit Rate: 0.8409785932721713
Average Long RoR: 0.02018012911159199
Long Investment: 56300
Long Return: 1136.141268982629
Average Short RoR: 0.0191057318212731
Short Investment: 41800
Short Return: 798.6195901292157
Total Investment: 98100
Total Return: 1934.7608591118446
Evaluating model predicting 5 days ahead using API images with 14 days timeframe.
30/30 [==============================] - 4s 125ms/step
Evaluation Metrics:
Accuracy: 0.7479166666666667
Precision: 0.7163120567375887
Recall: 0.831275720164609
F1 Score: 0.7695238095238096
Hit Rate: 0.7479166666666667
Average Long RoR: 0.01677323782257766
Long Investment: 56400
Long Return: 946.0106131933798
Average Short RoR: 0.015172205427891116
Short Investment: 39600
Short Return: 600.8193349444882
Total Investment: 96000
Total Return: 1546.82994813

In [ ]:
evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Long_Average_RoR', 'Long_Investment', 'Long_Return', 'Short_Average_RoR', 'Short_Investment', 'Short_Return', 'Total_Investment', 'Total_Return'])

TIMEFRAMES = [7, 14, 30, 90, 180]
PREDICTIONS = [5, 30, 90]

IMG_TYPES = ['MPA', 'CPA', 'MLC', 'API']

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction <= timeframe:
                print(f"Evaluating model predicting {prediction} days ahead using {img_type} images with {timeframe} days timeframe.")

                # Filter the data
                data = labels[(labels['TimePrediction'] == prediction) &
                              (labels['Image'].str.contains(f'/{img_type}/')) &
                              (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)

                # Start at same index as test data
                #split_index = int(len(data) * 0.8)
                #data = data[split_index:]

                # Load the model
                model_filename = f"models/noWeights_combined_{timeframe}_{prediction}.h5"
                model = load_model(model_filename)

                X = np.array(data['Image_Array'].tolist()) / 255.0
                lastPrice = data['LastPrice'].tolist()
                futurePrice = data['FuturePrice'].tolist()
                y = data['Label'].values

                # Evaluate the model on test data
                y_pred = model.predict(X)
                # Convert predictions to binary: if > 0.5 then 1 else 0
                y_pred_binary = np.where(y_pred > 0.5, 1, 0)  
                
                accuracy = accuracy_score(y, y_pred_binary)
                precision = precision_score(y, y_pred_binary)
                recall = recall_score(y, y_pred_binary)
                f1_score = fbeta_score(y, y_pred_binary, beta=1)

                y_test_array = y.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                hit_rate = correct_predictions / len(y_test_array)

                long_avg_ror, long_trades, long_investment, short_avg_ror, short_trades, short_investment = get_investment_return(y_pred_binary, lastPrice, futurePrice)
                # Total investment
                total_investment = long_investment + short_investment
                total_returns = long_trades + short_trades
                
                
                print("Evaluation Metrics:")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1_score}")
                print(f"Hit Rate: {hit_rate}")
                print(f"Average Long RoR: {long_avg_ror}")
                print(f"Long Investment: {long_investment}")
                print(f"Long Return: {long_trades}")
                print(f"Average Short RoR: {short_avg_ror}")
                print(f"Short Investment: {short_investment}")
                print(f"Short Return: {short_trades}")
                print(f"Total Investment: {total_investment}")
                print(f"Total Return: {total_returns}")

                
                # Add the evaluation metrics to the DataFrame
                evaluation_df = evaluation_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Accuracy': accuracy,
                    'Precision': precision,
                    'Recall': recall,
                    'F1_Score': f1_score,
                    'Hit_Rate': hit_rate,
                    'Long_Average_RoR': long_avg_ror,
                    'Long_Investment': long_investment,
                    'Long_Return': long_trades,
                    'Short_Average_RoR': short_avg_ror,
                    'Short_Investment': short_investment,
                    'Short_Return': short_trades, 
                    'Total_Investment': total_investment,
                    'Total_Return': total_returns
                }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
if TRANSFER:
    evaluation_df.to_csv('evaluation/combined/transfer_evaluation_scores.csv', index=False)
    print("Evaluation scores saved to 'evaluation/combined/tansfer_evaluation_scores.csv'.")
else:
    evaluation_df.to_csv('evaluation/combined/test_evaluation_scores.csv', index=False)
    print("Evaluation scores saved to 'evaluation/combined/test_evaluation_scores.csv'.")

In [19]:
# Get baseline RoR for each image type, timeframe, and prediction

ror_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Average_RoR'])

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction <= timeframe:

                # Filter your data based on prediction, img_type, and timeframe
                data = labels[(labels['TimePrediction'] == prediction) &
                              (labels['Image'].str.contains(f'/{img_type}/')) &
                              (labels['Image'].str.contains(f'_{timeframe}_'))]                
                data = data.reset_index(drop=True)
                #split_index = int(len(data) * 0.8)
                lastPrice = data['LastPrice']#[split_index:]
                # Last datapoint
                futurePrice = data['FuturePrice'].iloc[-1]
                print(futurePrice)

                #futurePrice = data['FuturePrice'][split_index:]
                # Calculate Rate of Return RoR
                ror = np.array((futurePrice - lastPrice)/lastPrice)
                # Calculate average RoR
                avg_ror = np.mean(ror)
                #print(f"Average RoR for {img_type} {timeframe} {prediction}: {avg_ror}")
                # Add the evaluation metrics to the DataFrame
                ror_df = ror_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Average_RoR': avg_ror
                }, ignore_index=True)

if TRANSFER:
    ror_df.to_csv('transfer_baseline_ror.csv', index=False)
else:
    ror_df.to_csv('test_baseline_ror.csv', index=False)

178.979996
178.130005
149.949997
161.130005
151.330002
146.229996
161.130005
146.839996
150.699997
156.580002
96.239998
161.759995
92.589996
100.339996
97.459999
91.519997
100.339996
98.610001
98.790001
98.870003
178.979996
162.119995
172.309998
184.910004
151.330002
146.229996
184.910004
146.839996
150.699997
156.580002
178.979996
178.130005
149.949997
161.130005
184.330002
146.229996
99.050003
98.360001
98.790001
156.580002


In [ ]:
# Overlay our three new images and evaluate the combined models on the newly crated dataset

evaluation_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Long_Average_RoR', 'Long_Investment', 'Long_Return', 'Short_Average_RoR', 'Short_Investment', 'Short_Return', 'Total_Investment', 'Total_Return'])

TIMEFRAMES = [7, 14, 30, 90, 180]
PREDICTIONS = [5, 30, 90]

IMG_TYPES = ['CPA', 'MLC', 'API']

if TRANSFER:
    etf_names = ["SmallCap", "Russell2000", "Semiconductor", "Healthcare", "NASDAQ", "DowJones", "CleanEnergy"]
else:
    etf_names = ["S&P500", "AI&Robotics", "MSCIWorld", "DAX", "EmergingMarkets", "Silver", "Treasury"]

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction <= timeframe:
            print(f"Evaluating model predicting {prediction} days ahead using overelayed images with {timeframe} days timeframe.")

            # Filter the data
            data = labels[(labels['TimePrediction'] == prediction) &
                            (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)

            # Start at same index as test data
            #split_index = int(len(data) * 0.8)
            #data = data[split_index:]

            # Build df with overlayed images
            overlayed_df = pd.DataFrame(columns=['Image_Array', 'LastPrice', 'FuturePrice', 'Label'])

            # For each timeframe, prediction and ETF overlay the images
            for date in data['Date'].unique():
                for etf in etf_names:
                    try:
                        # Filter the data
                        data_date = data[(data['Date'] == date) & (data['Image'].str.contains(f'{etf}'))]
                        # Load the images
                        img1 = data_date[data_date['Image'].str.contains('CPA')]['Image_Array'].values[0]
                        img2 = data_date[data_date['Image'].str.contains('MLC')]['Image_Array'].values[0]
                        img3 = data_date[data_date['Image'].str.contains('API')]['Image_Array'].values[0]
                        # Overlay the images
                        img = cv2.addWeighted(img1, 0.5, img2, 0.5, 0)
                        img = cv2.addWeighted(img, 0.5, img3, 0.5, 0)

                        # Save image to file
                        cv2.imwrite(f'images/overlayed/{etf}_{date}_{prediction}_{timeframe}.png', img)

                        # Add the image to the DataFrame
                        overlayed_df = overlayed_df.append({
                            'Image_Array': img,
                            'LastPrice': data_date['LastPrice'].values[0],
                            'FuturePrice': data_date['FuturePrice'].values[0],
                            'Label': data_date['Label'].values[0]
                        }, ignore_index=True)
                    except:
                        pass
            overlayed_df = overlayed_df.reset_index(drop=True)

            # Load the model
            model_filename = f"models/noWeights_combined_{timeframe}_{prediction}.h5"
            model = load_model(model_filename)

            X = np.array(overlayed_df['Image_Array'].tolist()) / 255.0
            lastPrice = overlayed_df['LastPrice'].tolist()
            futurePrice = overlayed_df['FuturePrice'].tolist()
            y = overlayed_df['Label'].values

            # Evaluate the model on test data
            y_pred = model.predict(X)
            # Convert predictions to binary: if > 0.5 then 1 else 0
            y_pred_binary = np.where(y_pred > 0.5, 1, 0)  

            y_list = list(y)
            y_pred_binary_list = list(y_pred_binary)

            y = np.array(y_list)
            y_pred_binary = np.array(y_pred_binary_list)       

            
            accuracy = accuracy_score(y, y_pred_binary)
            precision = precision_score(y, y_pred_binary)
            recall = recall_score(y, y_pred_binary)
            f1_score = fbeta_score(y, y_pred_binary, beta=1)

            y_test_array = y.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
            correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
            hit_rate = correct_predictions / len(y_test_array)
            print(hit_rate)

            long_avg_ror, long_trades, long_investment, short_avg_ror, short_trades, short_investment = get_investment_return(y_pred_binary, lastPrice, futurePrice)
            # Total investment
            total_investment = long_investment + short_investment
            total_returns = long_trades + short_trades
            
            
            print("Evaluation Metrics:")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1_score}")
            print(f"Hit Rate: {hit_rate}")
            print(f"Average Long RoR: {long_avg_ror}")
            print(f"Long Investment: {long_investment}")
            print(f"Long Return: {long_trades}")
            print(f"Average Short RoR: {short_avg_ror}")
            print(f"Short Investment: {short_investment}")
            print(f"Short Return: {short_trades}")
            print(f"Total Investment: {total_investment}")
            print(f"Total Return: {total_returns}")

            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1_Score': f1_score,
                'Hit_Rate': hit_rate,
                'Long_Average_RoR': long_avg_ror,
                'Long_Investment': long_investment,
                'Long_Return': long_trades,
                'Short_Average_RoR': short_avg_ror,
                'Short_Investment': short_investment,
                'Short_Return': short_trades, 
                'Total_Investment': total_investment,
                'Total_Return': total_returns
            }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
if TRANSFER:
    evaluation_df.to_csv('evaluation/overlayed/transfer_evaluation_scores.csv', index=False)
    print("Evaluation scores saved to 'evaluation/overlayed/tansfer_evaluation_scores.csv'.")
else:
    evaluation_df.to_csv('evaluation/overlayed/test_evaluation_scores.csv', index=False)
    print("Evaluation scores saved to 'evaluation/overlayed/test_evaluation_scores.csv'.")